# Collecting Bundestag data from Abgeordnetenwatch
> [Abgeordnetenwatch](https://www.abgeordnetenwatch.de) provides an [open API](https://www.abgeordnetenwatch.de/api) that provides info on, among other things, politicians, the politicians' votes and the different polls in parliament, including meta info.

This notebook collects the following information and prepares its parsing to `pandas.DataFrame` objects:
* polls for the 2017-2021 period of the Bundestag
* votes of members of the Bundestag 2017-2021
* info on members of the Bundestag 2017-2021

TODOs:
- identify why in vote json files some mandate_id values (politicians / mandates) appear multiple times (not always with the same vote result) -> affects `compile_votes_data` -> currently ignored and first of the duplicates used

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bundestag import abgeordnetenwatch as aw

In [ ]:
aw.ABGEORDNETENWATCH_PATH.parent

In [ ]:
aw.ABGEORDNETENWATCH_PATH.mkdir(exist_ok=True)

## Polls 2017-2021

Polls = objects voted on in the Bundestag by the parlamentarians

In [ ]:
dry = True # set `True` for testing, `False` otherwise

### Collecting

In [ ]:
legislature_id = 111
info = aw.get_poll_info(legislature_id, dry=dry)
aw.store_polls_json(info, legislature_id, dry=dry)

### Parsing

In [ ]:
legislature_id = 111
df = aw.get_polls_df(legislature_id)
aw.test_poll_data(df)
df.head()

In [ ]:
df.to_parquet(path=aw.ABGEORDNETENWATCH_PATH/'df_polls.parquet')

## Info on politicians

### Collecting

In [ ]:
legislature_id = 111
info = aw.get_mandates_info(legislature_id,dry=dry)
aw.store_mandates_info(info, legislature_id, dry=dry)

### Parsing

In [ ]:
legislature_id = 111
df = aw.get_mandates_df(legislature_id)
aw.test_mandate_data(df)
df['party'] = df.apply(aw.get_party_from_fraction_string, axis=1)
df.head().T

In [ ]:
df.to_parquet(path=aw.ABGEORDNETENWATCH_PATH/'df_mandates.parquet')

## Votes for one specific poll

### Collecting

In [ ]:
poll_id = 4217
info = aw.get_vote_info(poll_id, dry=dry)
aw.store_vote_info(info, poll_id, dry=dry)

### Parsing

In [ ]:
aw.test_stored_vote_ids_check()

In [ ]:
legislature_id, poll_id = 111, 4217
df = aw.get_votes_df(legislature_id, poll_id)
aw.test_vote_data(df)
df.head()

## All votes for all remaining polls of a specific legislative period

Above only one specific poll vote information was collected for. Here we collect votes for whatever polls are missing.

### Collecting

In [ ]:
legislature_id = 111
aw.get_all_remaining_vote_info(legislature_id, dry=dry)

### Parsing

In [ ]:
legislature_id = 111
df_all_votes = aw.compile_votes_data(legislature_id)

display(df_all_votes.head(), df_all_votes.tail())

Write compiled votes to disk as csv

In [ ]:
all_votes_path = aw.ABGEORDNETENWATCH_PATH / f'compiled_votes_legislature_{legislature_id}.csv'
df_all_votes.to_csv(all_votes_path, index=False)

In [ ]:
df_all_votes = df_all_votes.assign(
            **{"politician name": aw.get_politician_names}
        )
df_all_votes.to_parquet(path=aw.ABGEORDNETENWATCH_PATH/'df_all_votes.parquet')

In [ ]:
!head $all_votes_path